In [1]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch import nn
from transformers import BertTokenizer
from datasets import load_dataset

In [2]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [3]:
# Load IMDb dataset
dataset = load_dataset('imdb')

# Shuffle the dataset
dataset = dataset.shuffle(seed=42)

# Split the dataset
train_dataset = dataset['train'][:5000]  # Limiting the training size to 5000 samples
test_dataset = dataset['test'][:500]  # Limiting the test size to 200 samples

# Preprocess the text
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 4096  # Limiting the maximum length of tokens

def preprocess(text):
    tokens = tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    return tokens['input_ids'][0]

D:\UCSD Health Volunteer\Code\quantum-computing\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Quantum circuit
n_qubits = 5
dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev, interface='torch')
def quantum_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Quantum layer
class QuantumLayer(nn.Module):
    def __init__(self, n_qubits, n_layers):
        super(QuantumLayer, self).__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.weight_shapes = {"weights": (n_layers, n_qubits, 3)}
        self.qlayer = qml.qnn.TorchLayer(quantum_circuit, self.weight_shapes)

    def forward(self, x):
        return self.qlayer(x)

# Sentiment analysis model
class SentimentAnalysisModel(nn.Module):
    def __init__(self, n_qubits, n_layers):
        super(SentimentAnalysisModel, self).__init__()
        self.embedding = nn.Embedding(tokenizer.vocab_size, n_qubits)
        self.quantum_layer = QuantumLayer(n_qubits, n_layers)
        self.fc = nn.Linear(n_qubits, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x, dim=1)
        x = self.quantum_layer(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x


In [5]:
# Training the model
model = SentimentAnalysisModel(n_qubits, n_layers=2).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.2)

# Prepare training data
train_inputs = torch.stack([preprocess(text) for text in train_dataset['text']]).to(device)
train_labels = torch.tensor(train_dataset['label'], dtype=torch.float32).unsqueeze(1).to(device)


In [6]:
# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(train_inputs)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
    # Calculate training accuracy
    predictions = (outputs >= 0.5).float()
    correct_predictions = (predictions == train_labels).sum().item()
    train_accuracy = correct_predictions / len(train_labels)
    
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Train Accuracy: {train_accuracy * 100:.2f}%')

print("Training complete!")

Epoch 1/100, Loss: 0.698544442653656, Train Accuracy: 50.12%
Epoch 2/100, Loss: 0.7190716862678528, Train Accuracy: 49.88%
Epoch 3/100, Loss: 0.6933144927024841, Train Accuracy: 49.88%
Epoch 4/100, Loss: 0.6950789093971252, Train Accuracy: 50.12%
Epoch 5/100, Loss: 0.6962664723396301, Train Accuracy: 50.12%
Epoch 6/100, Loss: 0.6947662830352783, Train Accuracy: 50.12%
Epoch 7/100, Loss: 0.6929341554641724, Train Accuracy: 50.12%
Epoch 8/100, Loss: 0.6935946345329285, Train Accuracy: 49.88%
Epoch 9/100, Loss: 0.6932476162910461, Train Accuracy: 49.88%
Epoch 10/100, Loss: 0.6938587427139282, Train Accuracy: 50.12%
Epoch 11/100, Loss: 0.6933603286743164, Train Accuracy: 50.12%
Epoch 12/100, Loss: 0.6926799416542053, Train Accuracy: 49.88%
Epoch 13/100, Loss: 0.6925914287567139, Train Accuracy: 49.88%
Epoch 14/100, Loss: 0.6914692521095276, Train Accuracy: 49.88%
Epoch 15/100, Loss: 0.6907579302787781, Train Accuracy: 52.42%
Epoch 16/100, Loss: 0.6892753839492798, Train Accuracy: 54.36%
Ep

In [7]:
# Prepare test data
test_inputs = torch.stack([preprocess(text) for text in test_dataset['text']]).to(device)
test_labels = torch.tensor(test_dataset['label'], dtype=torch.float32).unsqueeze(1).to(device)

# Testing the model
model.eval()
with torch.no_grad():
    test_outputs = model(test_inputs)
    test_loss = criterion(test_outputs, test_labels)
    print(f'Test Loss: {test_loss.item()}')

    # Calculate accuracy
    predictions = (test_outputs >= 0.5).float()
    correct_predictions = (predictions == test_labels).sum().item()
    accuracy = correct_predictions / len(test_labels)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Loss: 0.40634748339653015
Test Accuracy: 83.80%


In [9]:
# Function to predict sentiment
def predict_sentiment(text, model):
    model.eval()
    with torch.no_grad():
        input_ids = preprocess(text).unsqueeze(0).to(device)  # Add batch dimension
        output = model(input_ids)
        prediction = output.item()
        sentiment = 'positive' if prediction >= 0.5 else 'negative'
        return sentiment, prediction

# Test the model on new data
test_texts = [
    "This movie was fantastic! I loved it.",
    "The film was terrible and a waste of time.",
    "An average movie with some good moments.",
    "Absolutely brilliant! A must-watch.",
    "Not my cup of tea. I didn't enjoy it.",
    "Absolutely worst!"
]

for text in test_texts:
    sentiment, score = predict_sentiment(text, model)
    print(f'Text: "{text}"\nSentiment: {sentiment} (Score: {score:.4f})\n')

Text: "This movie was fantastic! I loved it."
Sentiment: positive (Score: 0.6165)

Text: "The film was terrible and a waste of time."
Sentiment: negative (Score: 0.4318)

Text: "An average movie with some good moments."
Sentiment: positive (Score: 0.5120)

Text: "Absolutely brilliant! A must-watch."
Sentiment: positive (Score: 0.5694)

Text: "Not my cup of tea. I didn't enjoy it."
Sentiment: positive (Score: 0.5484)

Text: "Absolutely worst!"
Sentiment: negative (Score: 0.4772)

